contains all the code needed to prepare data

In [1]:
import requests
from bs4 import BeautifulSoup

import os
import json
import csv
import pandas as pd
import numpy as np

import time
from datetime import datetime

import bgg_data_func

# Games Master List from BGG

straightforward scraping

In [ ]:
session = requests.Session()
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)'\
           'AppleWebKit 537.36 (KHTML, like Gecko) Chrome',
           'Accept':'text/html,application/xhtml+xml,application/xml;'\
           'q=0.9,image/webp,*/*;q=0.8'}
url = 'https://boardgamegeek.com/browse/boardgame?sort=rank'
req = session.get(url, headers=headers)

bs = BeautifulSoup(req.text, 'html.parser')

In [ ]:
ranking_table = bs.find('table', {'class': 'collection_table'})

In [ ]:
rows = ranking_table.findChildren(['th', 'tr'])

In [ ]:
bg_name_list = []
bg_link_list = []
bg_year_list = []

for row in rows[8:]:
    cell = row.findChildren(['td'])[2]
    bg_name = cell.find('a').text
    bg_link = cell.find('a')['href']
    bg_year = cell.find('span', {'class': 'smallerfont dull'}).text[1:-1]
    
    bg_name_list.append(bg_name)
    bg_link_list.append(bg_link)
    bg_year_list.append(bg_year)
    
games_master_list = pd.DataFrame(columns=['name', 'link', 'year'])
games_master_list['name'] = bg_name_list
games_master_list['link'] = bg_link_list
games_master_list['year'] = bg_year_list

In [ ]:
games_master_list.to_csv('games_master_list.csv', index=False)

we have games_master_list.csv, with top 100 games, year, and link (which contains ID too)


prob wont have to run this often

# Import Master List

In [2]:
df = pd.read_csv('games_master_list.csv')

game_ids = df['link'].map(lambda x: bgg_data_func.link_to_gameid(x))

# BGG API Calls

in this section, looping through the 100 games


for each game, looping through the ratings


save ratings by 50 (100 doesnt work for some reason) in a json file in json_data folder

In [8]:
# cell saves all available pages for a given gamelist, change game_ids parameters
# works with other string IDs as well but must have a folder

for game_id in game_ids[35:40]:
    
    print(game_id)
    
    page_num = 1
    page_empty = False

    while not page_empty:

        page_empty = bgg_data_func.handle_one_api_request(game_id, page_num)

        seconds_to_sleep = 4 + np.random.uniform(0,1)
        time.sleep(seconds_to_sleep)
        
        page_num += 1
        
        if page_num % 50 == 0:
            print(page_num)
        
    time.sleep(10)

2651
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
209010
50
100
150
200
250
164153
50
100
150
200
250
300
350
126163
50
100
150
200
250
300
350
400
450
500
550
25613
50
100
150
200
250
300
350


In [7]:
# this one just gets info for one game, from a certain page num

game_id = '216132'
page_num = 1
rated_only = True

page_empty = False

while not page_empty:

    # get API response
    page_empty = bgg_data_func.handle_one_api_request(game_id, page_num, rated_only)

    seconds_to_sleep = 2 + np.random.uniform(0,1)
    time.sleep(seconds_to_sleep)

    page_num += 1
    
    if page_num % 50 == 0:
        print(page_num)

50
100
150
200


In [2]:
# and finally, this one is a given page num for a given game

game_id = '123260'
page_num = 1

bgg_data_func.handle_one_api_request(game_id, page_num, False)

False

# JSON Data Converter

this section takes the raw json files the API returns, converts them into usable csvs

In [9]:
game_ids_to_convert = game_ids[32:40]

In [10]:
# testing
for game_id in game_ids_to_convert:
    print(game_id)

221107
237182
167355
2651
209010
164153
126163
25613


In [11]:
for game_id in game_ids_to_convert:
    bgg_data_func.create_csv_summary(game_id)

# Create Data Summary

this section loops through the csv summary files per game and creates files that can be imported to different models. saves them in data_inputs folder

In [12]:
summary_filename = "games_40"

In [13]:
for game_id in game_ids[:40]:
    
    current_file = pd.read_csv('./json_data/game_' + game_id + '/' + game_id + '_summary.csv', header = None)
    
    current_file[2] = game_id
    
    with open('./data_input/' + summary_filename + '_summary.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerows(zip(current_file[0], current_file[2], current_file[1]))
    